In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
# label - 0: reliable, 1: unreliable

train_data = pd.read_csv('train.csv', dtype={'id': str}, index_col = 'id')
new_data = pd.read_csv('test.csv', dtype={'id': str}, index_col = 'id')

In [3]:
# separate features and labels

X = train_data[['author', 'title', 'text']]
y = train_data['label']

In [4]:
# initialize TF-IDF vectorizer

tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_df=0.7)

In [5]:
# create single feature column

X['combined_text'] = X.apply(lambda row: ' '.join(row.values.astype(str)), axis=1)

In [6]:
# split into train and test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
# fit and transform the training set

tfidf_train = tfidf_vectorizer.fit_transform(X_train['combined_text'])

In [8]:
# initiatelize and fit the classifier

pac = PassiveAggressiveClassifier(max_iter=50)
pac.fit(tfidf_train, y_train)

PassiveAggressiveClassifier(max_iter=50)

In [9]:
# create single test feature column

X_test['combined_text'] = X_test.apply(lambda row: ' '.join(row.values.astype(str)), axis=1)

In [10]:
# transform test set

tfidf_test = tfidf_vectorizer.transform(X_test['combined_text'])

In [11]:
# predict label in test set 

y_pred = pac.predict(tfidf_test)

In [12]:
# check the accuracy

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.9759615384615384


In [13]:
# prepare unlabeled data 

new_data['combined_text'] = new_data.apply(lambda row: ' '.join(row.values.astype(str)), axis=1)

In [14]:
# transorm unlabeled data

tfidf_new = tfidf_vectorizer.transform(new_data['combined_text'])

In [15]:
# make precitions on new data using model

new_data['predicted_label'] = pac.predict(tfidf_new)

In [25]:
# save labeled new data to new file 'update.csv'

new_data.to_csv('update.csv')

In [26]:
# use update.csv as an update to our model, assuming the labels are predicted correctly

update_data = pd.read_csv('update.csv')

In [27]:
# combine features

update_data['combined_text'] = update_data.apply(lambda row: ' '.join(row[['author', 'title', 'text']].values.astype(str)), axis=1)

In [28]:
# use tf-idf vectorizer

tfidf_update = tfidf_vectorizer.transform(update_data['combined_text'])

In [31]:
# update model with the new data

pac.partial_fit(tfidf_update, update_data['predicted_label'], classes=[0, 1])

PassiveAggressiveClassifier(max_iter=50)